In [13]:
import google.generativeai as genai
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv

# --- 1. 常數與提示詞定義 (來自您的 GRPO 腳本) ---

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""

EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"

In [14]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)



def extract_xml_answer(text: str) -> str:
    """提取 <answer> 標籤內的內容"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    return answer


def is_text(text: str) -> bool:
    """判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）"""
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)
    ratio = letters_and_spaces / max(1, len(text))  
    return ratio > 0.5


def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    return f1



def count_xml(text: str) -> float:
    """
    檢查 XML 標籤格式的完整性
    
    Returns:
        分數 (0.0 到 0.5),分數越高表示格式越完整
    """
    score = 0.0
    
    if text.count("<reasoning>\n") == 1:
        score += 0.125
    if text.count("\n</reasoning>\n") == 1:
        score += 0.125
    if text.count("\n<answer>\n") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>\n")[-1])
        score -= extra_content * 0.001
    if text.count("\n</answer>") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>")[-1]) - 1
        score -= max(0, extra_content) * 0.001
    
    return max(0.0, score)  # ✅ 確保不會是負數

In [ ]:
# --- 4. 載入 Gemini 模型 ---
# set GOOGLE_API_KEY="YOUR_API_KEY_HERE"
# api_key = os.getenv("GOOGLE_API_KEY")  # 修正：從環境變數讀取
# if not api_key:
#     print("❌ 錯誤：請設定 'GOOGLE_API_KEY' 環境變數。")
#     print("   在 Linux/Mac 上執行: export GOOGLE_API_KEY='your_api_key'")
#     print("   在 Windows 上執行: set GOOGLE_API_KEY=your_api_key")
#     exit()
    
# genai.configure(api_key=api_key)

genai.configure(api_key=" ")

print(f"✅ 正在設定模型: gemini-2.5-pro ...")

# 建立模型實例，包含 System Prompt
model = genai.GenerativeModel(
    model_name="gemini-2.5-pro",
    system_instruction=SYSTEM_PROMPT
)

print("✅ 模型設定完畢。")

# --- 5. 載入並處理資料 ---
print("\n📂 正在載入 note_icd_data.jsonl ...")
chat_style_data = []
MAX_RECORDS_TO_PROCESS = 714  # 限制最大處理筆數

try:
    with open("note_icd_data.jsonl", "r", encoding="utf-8") as f:
        lines = f.readlines()
        
        if len(lines) > MAX_RECORDS_TO_PROCESS:
            print(f"📊 資料共 {len(lines)} 筆，僅讀取前 {MAX_RECORDS_TO_PROCESS} 筆進行測試。")
            lines = lines[:MAX_RECORDS_TO_PROCESS]
        else:
            print(f"📊 資料共 {len(lines)} 筆。")

        for line in tqdm(lines, desc="[Gemini] 載入並預處理資料"):
            item = json.loads(line)
            ground_truth_answer = extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })

except FileNotFoundError:
    print("❌ 錯誤：找不到 'note_icd_data.jsonl' 檔案。")
    exit()

print(f"✅ 資料載入完畢，共 {len(chat_style_data)} 筆。\n")

✅ 正在設定模型: gemini-2.5-pro ...
✅ 模型設定完畢。

📂 正在載入 note_icd_data.jsonl ...
📊 資料共 3852 筆，僅讀取前 714 筆進行測試。


[Gemini] 載入並預處理資料: 100%|██████████| 714/714 [00:00<00:00, 43869.87it/s]

✅ 資料載入完畢，共 714 筆。



In [16]:
# --- 6. 執行推論並計算指標 ---
all_predictions = []
all_ground_truths = []
all_f1_scores = []
all_completion_lengths = []

# 初始化 CSV 檔案
csv_filename = "gemini_pro_results_v0.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    print(f"📁 CSV 檔案已初始化: {csv_filename}\n")
    
    # 開始推論
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Gemini] 執行推論與評估"), 1):
        try:
            question = item["raw_question"]
            gt_answer = item["answer"]
            
            # 建立包含 one-shot 範例的完整對話
            chat = model.start_chat(history=[
                {"role": "user", "parts": [EXAMPLE_QUESTION]},
                {"role": "model", "parts": [f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"]}
            ])
            
            # 發送問題並取得回應
            response = chat.send_message(question)
            assistant_reply = response.text
            
            # 印出完整資訊
            print("\n" + "="*80)
            print(f"📝 樣本 #{idx}")
            print("="*80)
            print(f"\n🤖 模型完整回答:")
            print("-"*80)
            print(assistant_reply)
            print("-"*80)
            
            # 記錄 ground truth
            all_ground_truths.append(gt_answer)
            
            # 計算 completion length (字元數，因為 Gemini API 不直接返回 token)
            completion_length = len(assistant_reply)
            print(f"\n📊 完成的回覆長度 (字元數): {completion_length}")
            all_completion_lengths.append(completion_length)
            
            # 提取預測的 ICD 編碼
            predicted_codes = extract_icd_answer(assistant_reply)
            print(f"🔍 提取的 ICD-10 編碼: {predicted_codes}")
            all_predictions.append(predicted_codes)
            
            print(f"\n🎯 Ground Truth Answer:")
            print(f"   {gt_answer}")
            
            # 計算 F1 分數
            f1_score = compute_f1(predicted_codes, gt_answer)
            print(f"✅ F1 分數: {f1_score:.4f}")
            all_f1_scores.append(f1_score)
            
            # 寫入 CSV
            writer.writerow({
                'accuracy_reward': f1_score,     
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer if isinstance(gt_answer, str) else json.dumps(gt_answer), 
                'predictions_ans': json.dumps(predicted_codes) if isinstance(predicted_codes, list) else predicted_codes 
            })
            csv_file.flush()
            
            print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
            print("="*80 + "\n")
            
        except Exception as e:
            print(f"\n❌ 處理第 {idx} 筆資料時發生錯誤:")
            print(f"   錯誤訊息: {e}")
            break
            # print(f"   跳過此筆資料繼續執行...\n")
            # continue

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 7. 顯示最終統計 ---
if not all_predictions:
    print("⚠️  沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print("  Gemini-2.5-pro 最終統計")
    print("="*50)
    print(f"✅ 總共處理: {len(all_f1_scores)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_f1_scores)/len(all_f1_scores):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} 字元")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已初始化: gemini_pro_results_v0.csv



[Gemini] 執行推論與評估:   0%|          | 1/714 [00:44<8:42:54, 44.00s/it]


📝 樣本 #1

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **主要症狀與原因**: 患者主訴為虛弱和腹瀉。病史表明，腹瀉是為了治療鴉片類藥物引起的便秘而服用「天然瀉藥」所致。這屬於藥物引起的非感染性腹瀉，應編碼為 **K52.1 (毒性胃腸炎和結腸炎)**。
2.  **脫水**: 腹瀉導致了脫水，實驗室檢查顯示 BUN/Cr 比例升高 (36/1.1)，這是腎前性氮質血症的典型表現，支持脫水的診斷。應編碼為 **E86.0 (脫水)**。
3.  **貧血**: 患者有「慢性貧血」病史，此次入院時血紅素降至 7.8 g/dL，低於基線，需要輸血治療。鐵蛋白數值 (249) 偏高，排除了缺鐵性貧血，更支持「慢性病性貧血」。應編碼為 **D63.8 (其他慢性疾病引起的貧血)**。
4.  **骨質疏鬆症與骨折**: 患者有「骨質疏鬆症」和近期因跌倒導致的「急性 L5 骨折」。在骨質疏鬆背景下的輕微創傷（如站立時跌倒）所致的骨折應被視為病理性骨折。最精確的編碼是 **M80.08xA (與年齡相關的骨質疏鬆症伴有現時的病理性骨折，椎骨，初次診療)**，此編碼已同時涵蓋骨質疏鬆和病理性骨折。
5.  **既往病史**:
    *   **高血壓 (Hypertension)**: 一項明確的慢性病史，應編碼為 **I10 (原發性高血壓)**。
    *   **共濟失調 (Ataxia)**: 一項明確的神經系統病史，與其反覆跌倒的風險相關，應編碼為 **R27.0 (共濟失調，未特指)**。
    *   **反覆跌倒 (Recurrent falls)**: 病史中明確提及，是一個重要的臨床問題，應編碼為 **R29.6 (反覆跌倒)**。
6.  **藥物不良反應**:
    *   病情的起因是鴉片類藥物導致的便秘。這是一種藥物不良反應。便秘本身應編碼為 **K59.00 (便秘，未特指)**。
    *   引起便秘的鴉片類藥物應使用外部原因碼來說明，編碼為 **T40.2X5A (其他鴉片類藥物的不良反應，初次診療)**。
    *   引起腹瀉的瀉藥同樣是藥

[Gemini] 執行推論與評估:   0%|          | 2/714 [01:12<6:54:37, 34.94s/it]


📝 樣本 #2

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **R57.0 - 心因性休克 (Cardiogenic shock)**：這是患者的主訴，也是轉院的主要原因。病歷中明確提到「in cardiogenic shock」，且患者需要使用升壓藥物 Levophed 來維持血壓，這都直接支持此診斷。
2.  **I50.23 - 急性發作於慢性收縮性 (鬱血性) 心臟衰竭 (Acute on chronic systolic (congestive) heart failure)**：患者有多次因心臟衰竭入院的病史，此次表現為呼吸困難、下肢水腫惡化，BNP升高，這些都是心衰竭急性發作的典型症狀。其射出分率 (EF) 僅 25% 明確指出其為收縮性心衰竭。因此，此編碼準確描述了心因性休克的根本原因。
3.  **I25.5 - 缺血性心肌病 (Ischemic cardiomyopathy)**：病史中明確記載患者患有「ischemic cardiomyopathy with EF 25%」。這是導致其慢性心臟衰竭和低射出分率的根本病因。
4.  **I25.10 - 動脈粥樣硬化性心臟病，未伴有心絞痛 (Atherosclerotic heart disease of native coronary artery without angina pectoris)**：病歷提及患者有「3 vessel CAD s/p RCA stenting」（三支冠狀動脈疾病並曾接受右冠狀動脈支架置放術），這是其缺血性心肌病的基礎。I25.10 是對這種慢性冠狀動脈疾病的標準編碼。
5.  **I48.91 - 未明示之心房顫動 (Unspecified atrial fibrillation)**：病歷中提到，患者在使用 Dobutamine 後出現了心房顫動 (afib) 和快速心律不整 (tachyarrhythmia)，這是一個在住院期間發生的重要併發症。
6.  **E11.22 - 第2型糖尿病伴有糖尿病慢性腎臟病 (Type 2 diabetes mellitus with di

...


📝 樣本 #342

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **主要診斷 - 中毒與精神狀態改變**：患者主訴為「AMS」（精神狀態改變），病史中明確提到「I took too much klonopin, and I passed out」（我吃了太多克諾平，然後昏倒了）。Klonopin（Clonazepam）是一種苯二氮䓬類藥物。這構成了意外中毒。精神科會診也懷疑「benzo intoxication, resolving delirium」（苯二氮䓬類藥物中毒，緩解中的譫妄）。
    *   `T42.4X1A`: Poisoning by benzodiazepines, accidental (unintentional), initial encounter (苯二氮䓬類藥物意外中毒，初次就診)。患者是為了睡覺而多吃，非自殺意圖，故歸類為意外。
    *   `F13.220`: Sedative, hypnotic, or anxiolytic use, moderate or severe, with intoxication delirium (鎮靜、安眠或抗焦慮藥物使用，中度或重度，伴有中毒性譫妄)。此編碼比單純的「AMS」(R41.82) 更精確地描述了由藥物中毒引起的譫妄狀態。

2.  **中毒導致的後果 - 跌倒與相關檢查**：患者因藥物過量而昏倒並跌倒（"passed out on the couch... and fell"）。
    *   `W08.XXXA`: Fall from other furniture, initial encounter (從其他家具跌落，初次就診)。患者是從沙發上起身時跌倒。
    *   儘管頭部、脊椎、腳踝和髖部的影像學檢查均為陰性，但跌倒是臨床上重要的事件，需要編碼。

3.  **潛在原因與相關診斷**：
    *   `G47.00`: Insomnia, unspecified (失眠，未特定)。患者明確表示多吃藥是「desperate for sleep」（極度渴望睡眠），並提到「Chr

[Gemini] 執行推論與評估:  48%|████▊     | 343/714 [2:59:47<3:45:07, 36.41s/it]


📝 樣本 #343

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **周邊 T 細胞淋巴瘤 (Peripheral T cell lymphoma)**：這是患者最主要的潛在診斷，並且是導致目前所有問題的根本原因。病歷中明確提到「with peripheral T cell lymphoma s/p CHOP x6 cycles with recurrent disease」。對應的 ICD-10 編碼為 **C84.40** (Peripheral T-cell lymphoma, not elsewhere classified, unspecified site)。
2.  **發燒 (Fever)**：這是患者的主訴 (Chief Complaint)，並且在就診時體溫為 100.8°F (38.2°C)。由於發燒與已知的惡性腫瘤和嗜中性白血球減少症有關，因此最適合的編碼是 **R50.81** (Fever presenting with conditions classified elsewhere)，這比 R50.9 (Fever, unspecified) 更具特異性。
3.  **化療引起的嗜中性白血球減少症 (Neutropenia due to chemotherapy)**：病歷中明確指出患者因嗜中性白血球減少症 (ANC 290) 而暫停了化療，並開始使用 Neupogen。這是本次入院評估發燒的關鍵背景。此情況對應的編碼是 **D70.1** (Agranulocytosis secondary to cancer chemotherapy)，這個編碼涵蓋了由化療引起的嚴重嗜中性白血球減少症。
4.  **因腫瘤疾病引起的貧血 (Anemia in neoplastic disease)**：實驗室檢查顯示 HCT 為 23.5，這表示患者有顯著的貧血。考慮到她的淋巴瘤診斷和正在接受的化療，這是預期中的併發症。對應的編碼是 **D63.0** (Anemia in neoplastic disease)。
5.  **續發性血小板減少症 (Secondary

[Gemini] 執行推論與評估:  48%|████▊     | 343/714 [2:59:50<3:14:31, 31.46s/it]


KeyboardInterrupt: 